# Challenge 2: Reverse Transactions Analyzer (Gemini Version)

This notebook implements an AI solution for the "Reverse Transactions" category of the "Strengthening the Adoption of Standards in Islamic Finance with Artificial Intelligence" Hackathon.

## Objective
Given "out-of-context" financial entries (journal entries and brief context), our AI solution will identify the relevant AAOIFI Financial Accounting Standard(s) (FAS) that govern such transactions. If multiple FAS are possible, the system provides a weighted probability and reasoning.

## Approach
1. Process and index the AAOIFI standards (FAS and SS) using RAG techniques with Google's Gemini embedding model
2. Create a specialized prompt engineering system for reverse transaction analysis 
3. Provide weighted probability estimations for applicable standards
4. Justify the analysis with references to specific sections of the standards

In [8]:
# Import required libraries
import os
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
import re
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

# Load environment variables
load_dotenv()

# Check if GOOGLE_API_KEY is set
if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY environment variable is not set. Please set it to use Gemini models.")

## Step 1: Data Extraction from FAS and SS Standards
First, we'll extract text from the relevant AAOIFI standards documents (FAS and SS). These documents contain the rules and guidelines that determine the appropriate accounting treatments for various Islamic finance transactions.

In [9]:
# Function to extract text from PDF documents
def extract_pdf_text(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return ""

# Get all PDF files from the data directory
data_dir = "../data"
pdf_files = [f for f in os.listdir(data_dir) if f.endswith('.pdf') or f.endswith('.PDF')]

# Extract text from each PDF
documents = []
for pdf_file in pdf_files:
    pdf_path = os.path.join(data_dir, pdf_file)
    text = extract_pdf_text(pdf_path)
    if text:
        documents.append({
            "source": pdf_file,
            "text": text
        })
        print(f"Successfully extracted text from {pdf_file}")
    else:
        print(f"Failed to extract text from {pdf_file}")

print(f"\nTotal documents processed: {len(documents)}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Successfully extracted text from FAS 10 & SS 11.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Successfully extracted text from FAS10.PDF


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Successfully extracted text from FAS28.PDF


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Successfully extracted text from FAS32.pdf
Successfully extracted text from FAS4.PDF
Successfully extracted text from FAS4.PDF
Successfully extracted text from FAS7.PDF
Successfully extracted text from FAS7.PDF
Successfully extracted text from SS10.pdf
Successfully extracted text from SS10.pdf
Successfully extracted text from SS12.pdf
Successfully extracted text from SS8.pdf
Successfully extracted text from SS9.pdf

Total documents processed: 10


## Step 2: Text Preprocessing and Chunking
To optimize the retrieval process, we need to divide the document text into smaller, meaningful chunks. This enables more precise matching and retrieval when handling specific queries about accounting standards.

In [10]:
# Initialize text splitter for chunking
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Larger chunks to capture more context
    chunk_overlap=200,  # Significant overlap to preserve context across chunks
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Process documents and create chunks with metadata
chunks = []
for doc in documents:
    for i, chunk in enumerate(splitter.split_text(doc["text"])):
        # Extract standard number from filename if possible
        standard_match = re.search(r'(FAS|SS)[\s_-]*(\d+)', doc["source"], re.IGNORECASE)
        standard_type = standard_match.group(1).upper() if standard_match else "Unknown"
        standard_number = standard_match.group(2) if standard_match else "Unknown"
        
        chunks.append({
            "source": doc["source"],
            "text": chunk,
            "chunk_id": i,
            "standard_type": standard_type,
            "standard_number": standard_number
        })

print(f"Total chunks created: {len(chunks)}")

Total chunks created: 1130


## Step 3: Building the Vector Database with Gemini Embeddings
Now we'll create a vector database using Google's Gemini embeddings to store and retrieve our document chunks. This enables semantic search capabilities, allowing us to find the most relevant standard sections for a given financial transaction.

In [11]:
# Define path for Chroma persistence
CHROMA_PATH = "../vector_db/standards_reverse_transactions_gemini"

# Initialize Gemini embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"  # Gemini embedding model
)

# Extract chunk texts and prepare metadata
chunk_texts = [chunk["text"] for chunk in chunks]
chunk_metadatas = [
    {
        "source": chunk["source"],
        "chunk_id": chunk["chunk_id"],
        "standard_type": chunk["standard_type"],
        "standard_number": chunk["standard_number"]
    } 
    for chunk in chunks
]

# Check if the vector store already exists
if os.path.exists(CHROMA_PATH):
    # Load existing vector store
    print("Loading existing Chroma vector store...")
    vector_store = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embeddings
    )
else:
    # Create a new vector store
    print("Creating new Chroma vector store...")
    vector_store = Chroma.from_texts(
        texts=chunk_texts,
        embedding=embeddings,
        metadatas=chunk_metadatas,
        persist_directory=CHROMA_PATH
    )
    # Persist the vector store to disk
    vector_store.persist()
    print(f"Saved vector store to {CHROMA_PATH}")
    
print(f"Vector store contains {vector_store._collection.count()} documents")

Loading existing Chroma vector store...


C:\Users\Tayeb Kahia\AppData\Local\Temp\ipykernel_19284\1155358187.py:25: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


Vector store contains 1130 documents


## Step 4: Retrieval Functions
Let's create functions to retrieve the most relevant standards for a given financial transaction. These functions will help us find the right context to determine which FAS applies to a particular journal entry.

In [12]:
def retrieve_relevant_standards(query, top_k=5):
    """
    Retrieve the most relevant chunks from our vector store for a given query.
    
    Args:
        query (str): The text query about a financial transaction
        top_k (int): Number of results to retrieve
        
    Returns:
        list: List of document chunks with metadata
    """
    # Use Chroma's similarity search to find relevant chunks
    docs = vector_store.similarity_search(query, k=top_k)
    
    # Convert the returned documents to our expected format
    results = []
    for doc in docs:
        results.append({
            "source": doc.metadata["source"],
            "standard_type": doc.metadata["standard_type"],
            "standard_number": doc.metadata["standard_number"],
            "text": doc.page_content
        })
    return results

def filter_by_standard(results, standard_type=None, standard_number=None):
    """
    Filter retrieved results by standard type and/or number.
    
    Args:
        results (list): List of document chunks
        standard_type (str, optional): Filter by standard type (FAS, SS)
        standard_number (str, optional): Filter by standard number
        
    Returns:
        list: Filtered list of document chunks
    """
    filtered = results
    
    if standard_type:
        filtered = [r for r in filtered if r["standard_type"].upper() == standard_type.upper()]
    
    if standard_number:
        filtered = [r for r in filtered if r["standard_number"] == standard_number]
        
    return filtered

## Step 5: FAS Identification and Weighting using Gemini
The core of our solution is the ability to identify which AAOIFI standards are most relevant to a given financial transaction. We'll use Google's Gemini model to analyze the transaction details and provide weighted probabilities for each potentially applicable standard.

In [13]:
def analyze_transaction(transaction_description, journal_entry=None, top_k=10):  # Increased top_k for more comprehensive context
    """
    Analyze a financial transaction and identify relevant AAOIFI standards using Gemini.
    
    Args:
        transaction_description (str): Description of the transaction
        journal_entry (str, optional): Journal entry related to the transaction
        top_k (int): Number of relevant chunks to retrieve
        
    Returns:
        dict: Analysis results with weighted probabilities
    """
    # Construct a comprehensive query combining description and journal entry
    query = transaction_description
    if journal_entry:
        query += f"\n{journal_entry}"
    
    # Enhanced query construction with explicit accounting scenarios to improve retrieval
    enhanced_query = f"""
    Financial transaction analysis:
    {query}
    
    Considering all possible accounting implications including:
    - Ownership changes and consolidation requirements
    - Contract modifications and reversals
    - Revenue and cost recognition
    - Asset recognition and classification
    - Shariah compliance considerations
    
    Relevant AAOIFI Financial Accounting Standards (FAS) and Shariah Standards (SS)
    """
    
    # Retrieve relevant chunks with increased top_k for more context
    relevant_chunks = retrieve_relevant_standards(enhanced_query, top_k=top_k)
    
    # Filter to focus on FAS documents
    fas_chunks = filter_by_standard(relevant_chunks, standard_type="FAS")
    
    # Extract unique FAS numbers for consideration
    potential_standards = []
    seen_standards = set()
    
    for chunk in fas_chunks:
        standard_id = f"FAS {chunk['standard_number']}"
        if standard_id not in seen_standards and chunk['standard_number'] != "Unknown":
            seen_standards.add(standard_id)
            potential_standards.append({
                "id": standard_id,
                "context": chunk['text'][:500]  # Brief context from the standard
            })
    
    # Key FAS standards to consider explicitly for all transactions
    # This ensures important standards are considered even if not returned by vector search
    key_fas_standards = ["FAS 1", "FAS 2", "FAS 4", "FAS 10", "FAS 20", "FAS 28", "FAS 32"]
    
    for standard in key_fas_standards:
        if standard not in seen_standards:
            # Add important standards even if not retrieved
            potential_standards.append({
                "id": standard,
                "context": f"This is a key AAOIFI standard that should be considered for completeness."
            })
            seen_standards.add(standard)
    
    # Also include SS standards as they often contain complementary guidance
    ss_chunks = filter_by_standard(relevant_chunks, standard_type="SS")
    ss_references = []
    
    for chunk in ss_chunks:
        standard_id = f"SS {chunk['standard_number']}"
        if standard_id not in seen_standards and chunk['standard_number'] != "Unknown":
            seen_standards.add(standard_id)
            ss_references.append({
                "id": standard_id,
                "context": chunk['text'][:300]  # Brief context from the standard
            })
    
    # Combine all relevant contexts for the LLM
    context_text = ""
    for std in potential_standards:
        context_text += f"Standard {std['id']}:\n{std['context']}\n\n"
    
    for std in ss_references:
        context_text += f"Supporting standard {std['id']}:\n{std['context']}\n\n"
    
    # Create Gemini LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0,
        convert_system_message_to_human=True
    )
    
    # Create enhanced prompt template with better guidance for standard selection
    prompt = PromptTemplate(
        input_variables=["transaction", "journal_entry", "context", "standard_ids"],
        template="""
    You are an expert in Islamic finance, AAOIFI standards, and accounting. You are tasked with analyzing financial transactions against AAOIFI standards.

    Transaction description:
    {transaction}
    
    Journal entry (if provided):
    {journal_entry}
    
    Excerpts from potentially relevant standards:
    {context}
    
    Potential standards to consider: {standard_ids}
    
    IMPORTANT ANALYSIS GUIDELINES:
    1. Consider BOTH direct AND downstream implications of transactions (e.g., 100% ownership necessitates consolidation)
    2. For ownership changes, particularly consider FAS 4 (Consolidation) and FAS 20 (Associates) when appropriate
    3. For contract modifications or reversals, identify the core contract type (Istisna'a, Ijarah, Murabaha, etc.)
    4. Look for key trigger phrases that indicate specific standards:
       - 100% ownership → consolidation (FAS 4, FAS 20)
       - Manufacturing/construction contracts → Istisna'a (FAS 10)
       - Leasing → Ijarah (FAS 32)
       - Deferred payment sales → Murabaha (FAS 28)
    5. Assign accurate probabilities based on relevance to the specific transaction details
    
    Based on your analysis, provide the following in JSON format:
    1. The FAS standards that apply to this transaction, with probability weights (0-100) totaling 100%
    2. A brief reasoning for each standard's applicability or inapplicability
    3. A determination if the journal entry appears to comply with the identified standards
    4. Any relevant Shariah considerations from the SS standards
    
    Return only the JSON object without additional commentary. Example format:
    {{
      "applicable_standards": [
        {{
          "standard": "FAS 28",
          "probability": 70,
          "reasoning": "This standard applies because..."
        }},
        {{
          "standard": "FAS 30",
          "probability": 30,
          "reasoning": "This standard is somewhat relevant because..."
        }}
      ],
      "compliance_assessment": "The journal entry appears to comply with FAS 28 because...",
      "shariah_considerations": "According to SS 9, this transaction should also consider..."
    }}
    """
    )
    
    # Prepare standard IDs for the prompt
    standard_ids = ", ".join([std["id"] for std in potential_standards])
    
    # Run the analysis
    chain = prompt | llm
    
    # Invoke the chain
    response = chain.invoke({
        "transaction": transaction_description,
        "journal_entry": journal_entry if journal_entry else "No journal entry provided",
        "context": context_text,
        "standard_ids": standard_ids
    })
    
    # Extract content from response
    if hasattr(response, 'content'):
        response_text = response.content
    else:
        response_text = str(response)
    
    # Parse the JSON output
    try:
        result = json.loads(response_text)
        
        # Sort applicable standards by probability in descending order and filter out 0 probability standards
        if "applicable_standards" in result:
            result["applicable_standards"] = [
                standard for standard in result["applicable_standards"] 
                if standard.get("probability", 0) > 0
            ]
            result["applicable_standards"].sort(key=lambda x: x.get("probability", 0), reverse=True)
        
        return result
    except json.JSONDecodeError:
        # If parsing fails, try to extract JSON from the response
        match = re.search(r'({.*})', response_text, re.DOTALL)
        if match:
            try:
                json_result = json.loads(match.group(1))
                
                # Sort applicable standards by probability in descending order and filter out 0 probability standards
                if "applicable_standards" in json_result:
                    json_result["applicable_standards"] = [
                        standard for standard in json_result["applicable_standards"] 
                        if standard.get("probability", 0) > 0
                    ]
                    json_result["applicable_standards"].sort(key=lambda x: x.get("probability", 0), reverse=True)
                
                return json_result
            except:
                pass
        
        # Return error if parsing fails
        return {
            "error": "Failed to parse LLM response",
            "raw_response": response_text
        }

## Step 6: Testing with Example Cases
Let's test our implementation with the example cases provided in the hackathon challenge:

### Example 1: GreenTech Exit and Buyout
- Context: GreenTech exits in Year 3, and Al Baraka Bank buys out its stake
- Adjustments: Buyout Price: $1,750,000; Bank Ownership: 100%
- Journal Entry: Dr. GreenTech Equity $1,750,000 / Cr. Cash $1,750,000

### Example 2: Contract Reversal
- Context: Client cancels a change order, reverting to original contract terms
- Adjustments: Revised Contract Value back to $5,000,000; Timeline Restored: 2 years
- Journal Entry: Dr. Accounts Payable $1,000,000 / Cr. Work-in-Progress $1,000,000

In [14]:
# Test Case 1: GreenTech Exit and Buyout
test_case_1 = {
    "description": """
    GreenTech exits in Year 3, and Al Baraka Bank buys out its stake. 
    After this buyout, the bank's ownership becomes 100%. 
    The accounting treatment involves derecognition of GreenTech's equity and recognition of the acquisition expense.
    """,
    "journal_entry": "Dr. GreenTech Equity $1,750,000 / Cr. Cash $1,750,000"
}

# Run the analysis
print("Analyzing Test Case 1: GreenTech Exit and Buyout\n")
result_1 = analyze_transaction(test_case_1["description"], test_case_1["journal_entry"])

# Display the results
print(json.dumps(result_1, indent=2))

Analyzing Test Case 1: GreenTech Exit and Buyout



d:\Anaconda3\envs\course-env\Lib\site-packages\langchain_google_genai\chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{
  "applicable_standards": [
    {
      "standard": "FAS 4",
      "probability": 70,
      "reasoning": "FAS 4 (Consolidation) is highly relevant as Al Baraka Bank now owns 100% of GreenTech.  This triggers the requirement for consolidating GreenTech's financial statements into Al Baraka's."
    },
    {
      "standard": "FAS 1",
      "probability": 10,
      "reasoning": "FAS 1 (Presentation and Disclosure) is generally applicable to all financial transactions, including this acquisition, as it sets the overall framework for financial reporting."
    },
    {
      "standard": "FAS 2",
      "probability": 10,
      "reasoning": "FAS 2 (Accounting for Specific Items) might be relevant depending on the specific nature of GreenTech's assets and liabilities being acquired.  Further details are needed to assess its full applicability."
    },
    {
      "standard": "FAS 28",
      "probability": 10,
      "reasoning": "FAS 28 (Murabaha and Other Deferred Payment Sales) is not direct

In [15]:
# Test Case 2: Contract Reversal
test_case_2 = {
    "description": """
    The client decided to cancel a change order that had previously modified the contract terms. 
    This cancellation reverts all terms back to the original contract. 
    The contract value is revised back to $5,000,000, and the project timeline is restored to the original 2 years. 
    This requires adjustment of revenue and cost projections, as well as a reversal of additional cost accruals.
    """,
    "journal_entry": "Dr. Accounts Payable $1,000,000 / Cr. Work-in-Progress $1,000,000"
}

# Run the analysis
print("Analyzing Test Case 2: Contract Reversal\n")
result_2 = analyze_transaction(test_case_2["description"], test_case_2["journal_entry"])

# Display the results
print(json.dumps(result_2, indent=2))

Analyzing Test Case 2: Contract Reversal



d:\Anaconda3\envs\course-env\Lib\site-packages\langchain_google_genai\chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{
  "applicable_standards": [
    {
      "standard": "FAS 10",
      "probability": 40,
      "reasoning": "If the original contract and the change order relate to construction or manufacturing, FAS 10 (Istisna'a) becomes highly relevant.  Reversing the change order would necessitate adjustments to the Istisna'a contract's accounting, including revenue and cost recognition."
    },
    {
      "standard": "FAS 1",
      "probability": 20,
      "reasoning": "This standard provides the general principles of accounting and should always be considered as a foundational standard for all transactions, including contract modifications."
    },
    {
      "standard": "FAS 2",
      "probability": 20,
      "reasoning": "This standard deals with accounting for inventories.  Given the reversal of costs related to the change order and its impact on work-in-progress, FAS 2 is relevant for the proper valuation and presentation of inventory affected by the contract revision."
    },
    {
      "